In [8]:
import pandas as pd
import numpy as np
import re

In [ ]:
movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")

In [ ]:
movies_df = movies_df.drop(57318)
movies_df = movies_df.drop(36122)

In [ ]:
df = movies_df.copy()

In [ ]:
df.sort_values(by = "year")

In [ ]:
list_1800 = list(df[df["year"]<1900]["movieId"])
list_1949 = list(df[(df["year"]>1899) & (df["year"]<1949)]["movieId"])
list_50s = list(df[(df["year"]>1949) & (df["year"]<1960)]["movieId"])
list_60s = list(df[(df["year"]>1959) & (df["year"]<1970)]["movieId"])
list_70s = list(df[(df["year"]>1969) & (df["year"]<1980)]["movieId"])
list_80s = list(df[(df["year"]>1979) & (df["year"]<1990)]["movieId"])
list_90s = list(df[(df["year"]>1989) & (df["year"]<2000)]["movieId"])
list_00s = list(df[(df["year"]>1999) & (df["year"]<2010)]["movieId"])
list_10s = list(df[(df["year"]>2009) & (df["year"]<2020)]["movieId"])

In [ ]:
def top10(list):
    list = ratings_df[ratings_df["movieId"].isin(list)]
    
    cnt = list.groupby(by="movieId",as_index= False).count().sort_values(by = "userId", ascending = False)
    cnt["vote_num"] = cnt["userId"]
    cnt = cnt.drop(columns = "rating")
    
    sum = list.copy()
    sum = sum.groupby(by= "movieId",as_index= False).sum()
    sum = sum[["movieId","rating"]]
    list = sum.merge(cnt,how="left",on= ("movieId"))
    list = list.drop(columns= ["timestamp","userId"])
    
    if(list["vote_num"].quantile(0.33) >=2000):
        list = list[list["vote_num"]>list["vote_num"].quantile(0.33)]
    
    else:
        list = list[list["vote_num"]>1999]
        
    list["recommend_rate"] =  list["rating"]/list["vote_num"]
    list = list.sort_values(by = "recommend_rate", ascending = False).head(20)
    l_m = list["movieId"]
    
    
    
    info = movies_df[movies_df['movieId'].isin(l_m)]
    list= list.reset_index(drop=True)
    list = list.merge(info, how = "left", on = "movieId")
    list = list[["movieId","title", "recommend_rate", "vote_num", "genres"]]

    return list

In [ ]:
top10(list_10s)

In [ ]:
HTML(top10(list_10s).to_html())